In [33]:
# features for Room Type One Hot Encoding 
expenses_features = ['RoomService', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck']
sel_features_cat = list(filter(lambda x: any(map(lambda y: y in x,['CryoSleep','HomePlanet', 'Destination'])), features_onehot))
features_onehot_wo_deck = list(filter(lambda x: 'Deck' not in x,list(features_onehot)))
onehot_deck_all = list(filter(lambda x: any(map(lambda y: y in x,['Deck'])), features_onehot))
onehot_deckB = list(filter(lambda x: ('Deck' not in x) or (x == 'Deck_B'), features_onehot))
onehot_deckBCEF = ['Deck_B','Deck_C','Deck_E','Deck_F','Deck_notBCEF']

# All features (drop Num and Name) [expenses features/total expenses] 
all_features_expenses = ['Age'] + expenses_features + list(features_onehot) 
all_features_totalexpenses = ['Age', 'TotalExpenses'] + list(features_onehot)
# Selection CryoSleep, HomePlanet, Deck, Destination, expenses features [expenses features/total expenses]
sel_features_expenses = expenses_features + sel_features_cat
sel_features_totalexpenses = ['TotalExpenses'] + sel_features_cat

# combinations of features to check 
#dataproc = [all_features_expenses, all_features_totalexpenses, sel_features_expenses, sel_features_totalexpenses]
dataproc = {'all_expenses': all_features_expenses, 'all_totalexpenses': all_features_totalexpenses, 
             'sel_expenses': sel_features_expenses, 'sel_totalexpenses': sel_features_totalexpenses}

In [16]:
X_train = df_train.drop('Transported', axis=1)
y_train = df_train['Transported']

In [29]:
# combinations of model to check (algorithm + parameters)
experiments = {
    'model': ['KNN', 'Ridge', 'DecisionTree', 'BaggingRidge', 'RandomForest']
    'algorithms': [KNeighborsClassifier(), RidgeClassifier(), 
                    DecisionTreeClassifier(criterion='gini'),
                    BaggingClassifier(estimator=RidgeClassifier())
                    RandomForestClassifier()],
    'parameters': [{'n_neighbors': [8, 10, 12, 14], 'weights': ['uniform', 'distance']},
                    {'alpha': [1.0]},
                    {'max_depth': [5, 10, 15], 'min_samples_split': [2, 5, 10], 'min_samples_leaf': [1, 2, 4], splitter['best', 'random']},
                    {'n_estimators': [10, 30, 50], 'max_features': [1.0, 0.7, 0.5], 'max_samples': [1.0, 0.9, 0.8]}
                    {'n_estimators': [50, 100, 200], 'max_depth': [5, 10, 15], 'min_samples_split': [2, 5, 10], 'min_samples_leaf': [1, 2, 4]}
                  ]
}

#experiments = {
#    'algorithms': [KNeighborsClassifier(), RidgeClassifier()],
#    'parameters': [{'n_neighbors': [14, 18], 'weights': ['uniform', 'distance']},
#                    {'alpha': [1.0]}
#                  ]
#}
#experiments = {
#    'model': ['KNN', 'Ridge'],
#    'algorithm': [KNeighborsClassifier(), RidgeClassifier()],
#    'parameters': [{'n_neighbors': [14, 18], 'weights': ['uniform', 'distance']},
#                    {'alpha': [1.0]}
#                  ]
#}
# run experiments: features + algorithm + parameters
results = []
for dp, features in dataproc.items():
    X_train_sel = X_train[features]    
    #for index, algorithm in enumerate(experiments['algorithms']):
    for index, model in enumerate(experiments['model']):
        params = experiments['parameters'][index]
        algorithm = experiments['algorithm'][index]
        grid = GridSearchCV(algorithm, param_grid = params, cv = 5, 
                            scoring= 'accuracy', return_train_score = True)
        grid.fit(X_train_sel, y_train)

        lst = list(grid.get_params()['param_grid'].values())
        #for params, mse in zip(itertools.product(*lst),grid.cv_results_['mean_test_score']):
        for params, test_score, train_score, fit_time in zip(itertools.product(*lst),
                                                   grid.cv_results_['mean_test_score'], 
                                                   grid.cv_results_['mean_train_score'],
                                                   grid.cv_results_['mean_fit_time']):
            params_dict = {param: value for param, value in zip(grid.get_params()['param_grid'].keys(), params)}
            result = {'feature_name': dp,'features': features, 'model': model, 'parameters': params_dict,
                  'train_score': train_score, 'test_score': test_score, 'fit_time': fit_time}
            results.append(result)


In [32]:
pd.DataFrame(results).drop(['features'],axis=1)
max_indices = pd.DataFrame(results).drop(['features'],axis=1).groupby('model')['test_score'].idxmax()
pd.DataFrame(results).drop(['features'],axis=1).loc[max_indices]


,feature_name,model,parameters,train_score,test_score,fit_time
2,all_expenses,KNN,"{'n_neighbors': 18, 'weights': 'uniform'}",0.808754,0.780285,0.004548
4,all_expenses,Ridge,{'alpha': 1.0},0.769642,0.767745,0.007219


In [57]:
best_model = pd.DataFrame(results).sort_values('acc_val', ascending=False).iloc[0,:].to_dict()
for key, value in best_model.items():
    print(f'{key}: {value}')

features: ['RoomService', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck', 'Destination_55 Cancri e', 'Destination_PSO J318.5-22', 'Destination_TRAPPIST-1e', 'HomePlanet_Earth', 'HomePlanet_Europa', 'HomePlanet_Mars', 'CryoSleep_True']
algorithm: DecisionTreeClassifier(splitter='random')
parameters: {'max_depth': 10, 'min_samples_split': 2, 'min_samples_leaf': 4}
acc_val: 0.7839655609383592
